In [3]:
from google.cloud import texttospeech

In [24]:
from functools import reduce

In [4]:
client = texttospeech.TextToSpeechClient.from_service_account_json("AutoPodcast-d352f15e5933.json")

In [53]:
with open("short_sample.txt", encoding="utf-8") as f:
    text = f.read()
len(text)

4306

In [5]:
def t2s(text):
    synthesis_input = texttospeech.SynthesisInput(ssml=text)
    voice = texttospeech.VoiceSelectionParams(
        name="en-US-Wavenet-A", language_code="en-US"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    return response.audio_content

In [6]:
def f2s(path):
    with open(path, encoding="utf-8") as f:
        return t2s(f.read())

In [91]:
files = ["part1.txt", "part2.txt", "part3.txt", "part4.txt"]

snippets = [f2s(p) for p in files]


In [8]:
# The response's audio_content is binary.
def save_snippet(s, path):
    with open(path, "wb") as out:
        # Write the response to the output file.
        out.write(s)

In [33]:
path = "ssml/The TechCrunch Exchange - Bootstrapping to $80M ARR.txt"
with open(path) as f:
    lines = f.readlines()
snippets = [t2s(l) for l in lines]

In [34]:
combined_file = reduce(lambda a, b: a + b, snippets)

In [35]:
save_snippet(combined_file, "Exchange.mp3")

In [37]:
from google.cloud import storage
import io

In [38]:
storage_client = storage.Client.from_service_account_json("AutoPodcast-d352f15e5933.json")

In [121]:
def upload_bytes(bucket_name, blob_name, content):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_file(io.BytesIO(content))
    return blob.public_url

In [122]:
upload_bytes("autopodcast", "podcast.mp3", combined_file)

'https://storage.googleapis.com/autopodcast/podcast.mp3'